In [29]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc

# MERGE OOF

### FTRL

In [30]:
FTRL_OOF_1 = pd.read_csv("../input/fm-ftrl-lb-0-9769-oof-part1/wordbatch_fm_ftrl.csv")
FTRL_OOF_2 = pd.read_csv("../input/fm-ftrl-lb-0-9769-oof-part2/wordbatch_fm_ftrl.csv")
FTRL_OOF_3 = pd.read_csv("../input/fm-ftrl-lb-0-9769-oof-part3/wordbatch_fm_ftrl.csv")
FTRL_OOF_4 = pd.read_csv("../input/fm-ftrl-lb-0-9769-oof-part4/wordbatch_fm_ftrl.csv")
FTRL_OOF = pd.concat([FTRL_OOF_1, FTRL_OOF_2[1:], FTRL_OOF_3, FTRL_OOF_4])
FTRL_OOF = FTRL_OOF.reset_index().drop(columns='index')
del FTRL_OOF_1, FTRL_OOF_2, FTRL_OOF_3, FTRL_OOF_4
gc.collect()

In [31]:
#FTRL_OOF

### LGB

In [32]:
LGB_OOF12 = pd.read_csv("../input/lgb-oof-12/LGB_OOF1.csv")
LGB_OOF34 = pd.read_csv("../input/lgb-oof-part34/LGB_OOF2.csv")
LGB_OOF = pd.concat([LGB_OOF12, LGB_OOF34[1:]])
LGB_OOF = LGB_OOF.reset_index().drop(columns='index')
del LGB_OOF12, LGB_OOF34
gc.collect()

In [33]:
#LGB_OOF

### Train

In [34]:
train = pd.read_csv("../input/talkingdata-adtracking-fraud-detection/train.csv",
                    usecols=['ip', 'is_attributed'],
                    nrows = 79999999)

### Merge

In [35]:
OOF = FTRL_OOF.join(LGB_OOF,lsuffix='_FTRL',rsuffix='_LGB')
del FTRL_OOF, LGB_OOF
gc.collect()

In [36]:
OOF = OOF.join(train)
del train
gc.collect()

In [37]:
OOF.drop(columns=['ip_FTRL', 'ip_LGB', 'ip'],inplace=True)

# Stacking

In [38]:
OOF = OOF[20000000:]

### Train

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.special import expit, logit

In [40]:
X_train = np.array(OOF[['is_attributed_FTRL','is_attributed_LGB']].apply(logit))
y_train = np.array(OOF['is_attributed'])
del OOF
gc.collect()

In [41]:
'''X, X_val, y, y_val = train_test_split(X_train, y_train, test_size=.33, shuffle=False )
del X_train, y_train
gc.collect()
for c in [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]:
    mod = LogisticRegression(C=c)
    mod.fit(X, y)
    val_pred = mod.predict_proba(X_val)[:,1]
    print( c, roc_auc_score(y_val, mod.predict_proba(X_val)[:,1]) )'''

In [42]:
stack_model = LogisticRegression(C=0.1)
stack_model.fit(X_train, y_train)
print(stack_model.coef_)
del X_train,y_train
gc.collect()

### Predict

In [43]:
subs = pd.read_csv('../input/sub-merge-2/subs.csv')
final_sub = pd.DataFrame()
final_sub['click_id'] = subs['click_id']
X_test = np.array(subs[['is_attributed_FTRL','is_attributed_LGB']].apply(logit))
del subs
gc.collect()

In [44]:
final_sub['is_attributed'] = stack_model.predict_proba(X_test)[:,1]
del X_test
gc.collect()

In [45]:
final_sub.to_csv("sub_stacked.csv", index=False, float_format='%.8f')

In [46]:
final_sub